In [527]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sqlite3 as sq
pd.options.mode.chained_assignment = None

In [528]:
def mantissa_round(x):
    y = np.floor(x)
    indices = np.argsort(x-y)
    hilowman = indices[::-1]
    diffxy = np.sum(x)-np.sum(y)
    z=0
    while z < int(diffxy):
        y[hilowman[z]] +=1
        z +=1
    return np.array(y, dtype='int')



In [529]:
dist = pd.read_csv('../data/LEHDDistribution.csv')
dist['County'] = dist['County'].str.split(pat = ',', expand = True)[0]
dist.head(2)

,GEO_ID,County,Total Jobs %,"Agriculture, forestry, fishing and hunting %","Mining, quarrying, and oil and gas extraction %",Utilities %,Construction %,Manufacturing %,Wholesale trade %,Retail trade %,Transportation and warehousing %,Information %,Finance and insurance %,Real estate and rental and leasing %,"Professional, scientific, and technical services %",Management of companies and enterprises %,Administrative and support and waste management and remediation services %,Educational services %,Health care and social assistance %,"Arts, entertainment, and recreation %",Accommodation and food services %,Other services (except government and government enterprises) %,Public Administration %
0,470210701021001,Cheatham County,0.009725,0.0,0.0,0.0,0.004242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.115318,0.0,0.0,0.0,0.020305
1,470210701021004,Cheatham County,0.002093,0.0,0.0,0.0,0.010604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022222,0.0,0.0,0.0,0.001721,0.0,0.0,0.0,0.000000


In [530]:
dist['GEO_ID'] = dist['GEO_ID'].astype(str)
c_inverted = dict(zip(dist['GEO_ID'], dist['County']))

In [531]:
bl = pd.read_csv('../data/mantissaadjbaseline.csv')
bl['County'] = bl['County'].str.split(pat = ',', expand = True)[0]
bl = bl.set_index('County')
bl = bl.add_suffix(' County Total').reset_index(drop = False)

In [532]:
bl.head(2)

,County,Accommodation and food services County Total,Administrative and support and waste management and remediation services County Total,"Agriculture, forestry, fishing and hunting County Total","Arts, entertainment, and recreation County Total",Construction County Total,Educational services County Total,Finance and insurance County Total,Health care and social assistance County Total,Information County Total,Management of companies and enterprises County Total,Manufacturing County Total,"Mining, quarrying, and oil and gas extraction County Total",Other services (except government and government enterprises) County Total,"Professional, scientific, and technical services County Total",Public Administration County Total,Real estate and rental and leasing County Total,Retail trade County Total,Transportation and warehousing County Total,Utilities County Total,Wholesale trade County Total,Total employment (number of jobs) County Total
0,Cheatham County,945,1061,491,593,2020,519,623,833,229,122,2961,17,1148,823,1209,913,1516,1052,57,236,17368
1,Davidson County,59595,63312,693,31535,42154,38447,45266,97834,22181,16622,22976,707,37087,65586,26262,40629,51863,50941,711,25933,740334


In [533]:
data = bl.merge(dist, on = 'County', how = 'outer')

In [534]:
industries = ['Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']
for ind in industries:
    data['{}'.format(ind)] = data['{} County Total'.format(ind)] * data['{} %'.format(ind)]

In [535]:
data.head(2)

,County,Accommodation and food services County Total,Administrative and support and waste management and remediation services County Total,"Agriculture, forestry, fishing and hunting County Total","Arts, entertainment, and recreation County Total",Construction County Total,Educational services County Total,Finance and insurance County Total,Health care and social assistance County Total,Information County Total,Management of companies and enterprises County Total,Manufacturing County Total,"Mining, quarrying, and oil and gas extraction County Total",Other services (except government and government enterprises) County Total,"Professional, scientific, and technical services County Total",Public Administration County Total,Real estate and rental and leasing County Total,Retail trade County Total,Transportation and warehousing County Total,Utilities County Total,Wholesale trade County Total,Total employment (number of jobs) County Total,GEO_ID,Total Jobs %,"Agriculture, forestry, fishing and hunting %","Mining, quarrying, and oil and gas extraction %",Utilities %,Construction %,Manufacturing %,Wholesale trade %,Retail trade %,Transportation and warehousing %,Information %,Finance and insurance %,Real estate and rental and leasing %,"Professional, scientific, and technical services %",Management of companies and enterprises %,Administrative and support and waste management and remediation services %,Educational services %,Health care and social assistance %,"Arts, entertainment, and recreation %",Accommodation and food services %,Other services (except government and government enterprises) %,Public Administration %,"Agriculture, forestry, fishing and hunting","Mining, quarrying, and oil and gas extraction",Utilities,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,Finance and insurance,Real estate and rental and leasing,"Professional, scientific, and technical services",Management of companies and enterprises,Administrative and support and waste management and remediation services,Educational services,Health care and social assistance,"Arts, entertainment, and recreation",Accommodation and food services,Other services (except government and government enterprises),Public Administration
0,Cheatham County,945,1061,491,593,2020,519,623,833,229,122,2961,17,1148,823,1209,913,1516,1052,57,236,17368,470210701021001,0.009725,0.0,0.0,0.0,0.004242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.115318,0.0,0.0,0.0,0.020305,0.0,0.0,0.0,8.568399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,96.060241,0.0,0.0,0.0,24.548223
1,Cheatham County,945,1061,491,593,2020,519,623,833,229,122,2961,17,1148,823,1209,913,1516,1052,57,236,17368,470210701021004,0.002093,0.0,0.0,0.0,0.010604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022222,0.0,0.0,0.0,0.001721,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,21.420997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.288889,0.0,0.0,0.0,1.433735,0.0,0.0,0.0,0.000000


In [536]:
data = data[['County', 'GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]

In [537]:
cheatham = data.loc[data['County'] == 'Cheatham County']
davidson = data.loc[data['County'] == 'Davidson County']
dickson = data.loc[data['County'] == 'Dickson County']
houston = data.loc[data['County'] == 'Houston County']
humphreys = data.loc[data['County'] == 'Humphreys County']
maury = data.loc[data['County'] == 'Maury County']
montgomery = data.loc[data['County'] == 'Montgomery County']
robertson = data.loc[data['County'] == 'Robertson County']
rutherford = data.loc[data['County'] == 'Rutherford County']
stewart = data.loc[data['County'] == 'Stewart County']
sumner = data.loc[data['County'] == 'Sumner County']
trousdale = data.loc[data['County'] == 'Trousdale County']
williamson = data.loc[data['County'] == 'Williamson County']
wilson = data.loc[data['County'] == 'Wilson County']

In [538]:
# dfs = [cheatham, davidson, dickson, houston, humphreys, maury, montgomery, robertson, rutherford, stewart, sumner, trousdale, williamson, wilson]
# inds = ['Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 'Manufacturing', 
#         'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 'Real estate and rental and leasing', 
#         'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#         'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#         'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#         'Public Administration']
# for df in dfs:
#     data = df
#     data = data[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
#               'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
#               'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#               'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#               'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#               'Public Administration']]
#     for ind in inds:
#         data['{} Mantissa'.format(ind)] = mantissa_round(data['{}'.format(ind)])
#         df = data

In [539]:
# cheatham = cheatham[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
#               'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
#               'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#               'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#               'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#               'Public Administration']]
# for ind in inds:
#     cheatham['{} Mantissa'.format(ind)] = mantissa_round(data['{}'.format(ind)])

In [540]:
cheatham = cheatham[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
cheatham.set_index('GEO_ID', inplace = True)
inds = list(cheatham.columns)
cheatham.reset_index(drop = False, inplace = True)
cheatham.fillna(0, inplace = True)
for ind in inds:
    cheatham['{} Mantissa'.format(ind)] = mantissa_round(cheatham['{}'.format(ind)])
cheatham['County'] = 'Cheatham County'

In [591]:
diff = cheatham.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff1 = diff.reset_index(drop = False)
diff1.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Cheatham County,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [542]:
davidson = davidson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
davidson.set_index('GEO_ID', inplace = True)
inds = list(davidson.columns)
davidson.reset_index(drop = False, inplace = True)
davidson.fillna(0, inplace = True)
for ind in inds:
    davidson['{} Mantissa'.format(ind)] = mantissa_round(davidson['{}'.format(ind)])
davidson['County'] = 'Davidson County'

In [590]:
diff = davidson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff2 = diff.reset_index(drop = False)
diff2.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Davidson County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [544]:
dickson = dickson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
dickson.set_index('GEO_ID', inplace = True)
inds = list(dickson.columns)
dickson.reset_index(drop = False, inplace = True)
dickson.fillna(0, inplace = True)
for ind in inds:
    dickson['{} Mantissa'.format(ind)] = mantissa_round(dickson['{}'.format(ind)])
dickson['County'] = 'Dickson County'

In [589]:
diff = dickson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff3 = diff.reset_index(drop = False)
diff3.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Dickson County,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [546]:
houston = houston[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
houston.set_index('GEO_ID', inplace = True)
inds = list(houston.columns)
houston.reset_index(drop = False, inplace = True)
houston.fillna(0, inplace = True)
for ind in inds:
    houston['{} Mantissa'.format(ind)] = mantissa_round(houston['{}'.format(ind)])
houston['County'] = 'Houston County'

In [588]:
diff = houston.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = (diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)])
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff4 = diff.reset_index(drop = False)
diff4.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Houston County,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-3.552714e-15,1.0,0.0,1.0,1.0,0.0


In [551]:
humphreys = humphreys[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
humphreys.set_index('GEO_ID', inplace = True)
inds = list(humphreys.columns)
humphreys.reset_index(drop = False, inplace = True)
humphreys.fillna(0, inplace = True)
for ind in inds:
    humphreys['{} Mantissa'.format(ind)] = mantissa_round(humphreys['{}'.format(ind)])
humphreys['County'] = 'Humphreys County'

In [587]:
diff = humphreys.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff5 = diff.reset_index(drop = False)
diff5.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Humphreys County,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [553]:
maury = maury[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
maury.set_index('GEO_ID', inplace = True)
inds = list(maury.columns)
maury.reset_index(drop = False, inplace = True)
maury.fillna(0, inplace = True)
for ind in inds:
    maury['{} Mantissa'.format(ind)] = mantissa_round(maury['{}'.format(ind)])
maury['County'] = 'Maury County'

In [586]:
diff = maury.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff6 = diff.reset_index(drop = False)
diff6.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Maury County,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [555]:
montgomery = montgomery[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
montgomery.set_index('GEO_ID', inplace = True)
inds = list(montgomery.columns)
montgomery.reset_index(drop = False, inplace = True)
montgomery.fillna(0, inplace = True)
for ind in inds:
    montgomery['{} Mantissa'.format(ind)] = mantissa_round(montgomery['{}'.format(ind)])
montgomery['County'] = 'Montgomery County'

In [585]:
diff = montgomery.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff7 = diff.reset_index(drop = False)
diff7.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Montgomery County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [557]:
robertson = robertson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
robertson.set_index('GEO_ID', inplace = True)
inds = list(robertson.columns)
robertson.reset_index(drop = False, inplace = True)
robertson.fillna(0, inplace = True)
for ind in inds:
    robertson['{} Mantissa'.format(ind)] = mantissa_round(robertson['{}'.format(ind)])
robertson['County'] = 'Robertson County'

In [584]:
diff = robertson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff8 = diff.reset_index(drop = False)
diff8.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Robertson County,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0


In [559]:
rutherford = rutherford[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
rutherford.set_index('GEO_ID', inplace = True)
inds = list(rutherford.columns)
rutherford.reset_index(drop = False, inplace = True)
rutherford.fillna(0, inplace = True)
for ind in inds:
    rutherford['{} Mantissa'.format(ind)] = mantissa_round(rutherford['{}'.format(ind)])
rutherford['County'] = 'Rutherford County'

In [583]:
diff = rutherford.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff9 = diff.reset_index(drop = False)
diff9.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Rutherford County,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [561]:
stewart = stewart[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
stewart.set_index('GEO_ID', inplace = True)
inds = list(stewart.columns)
stewart.reset_index(drop = False, inplace = True)
stewart.fillna(0, inplace = True)
for ind in inds:
    stewart['{} Mantissa'.format(ind)] = mantissa_round(stewart['{}'.format(ind)])
stewart['County'] = 'Stewart County'

In [582]:
diff = stewart.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff10 = diff.reset_index(drop = False)
diff10.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Stewart County,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,-7.105427e-15,1.0,0.0,1.0,1.0,1.0


In [563]:
sumner = sumner[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
sumner.set_index('GEO_ID', inplace = True)
inds = list(sumner.columns)
sumner.reset_index(drop = False, inplace = True)
sumner.fillna(0, inplace = True)
for ind in inds:
    sumner['{} Mantissa'.format(ind)] = mantissa_round(sumner['{}'.format(ind)])
sumner['County'] = 'Sumner County'

In [581]:
diff = sumner.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff11 = diff.reset_index(drop = False)
diff11.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Sumner County,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [565]:
trousdale = trousdale[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
trousdale.set_index('GEO_ID', inplace = True)
inds = list(trousdale.columns)
trousdale.reset_index(drop = False, inplace = True)
trousdale.fillna(0, inplace = True)
for ind in inds:
    trousdale['{} Mantissa'.format(ind)] = mantissa_round(trousdale['{}'.format(ind)])
trousdale['County'] = 'Trousdale County'

In [580]:
diff = trousdale.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff12 = diff.reset_index(drop = False)
diff12.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Trousdale County,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0


In [567]:
williamson = williamson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
williamson.set_index('GEO_ID', inplace = True)
inds = list(williamson.columns)
williamson.reset_index(drop = False, inplace = True)
williamson.fillna(0, inplace = True)
for ind in inds:
    williamson['{} Mantissa'.format(ind)] = mantissa_round(williamson['{}'.format(ind)])
williamson['County'] = 'Williamson County'

In [579]:
diff = williamson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff13 = diff.reset_index(drop = False)
diff13.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Williamson County,1.0,-5.684342e-14,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [569]:
wilson = wilson[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
              'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
              'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
              'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
              'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
              'Public Administration']]
wilson.set_index('GEO_ID', inplace = True)
inds = list(wilson.columns)
wilson.reset_index(drop = False, inplace = True)
wilson.fillna(0, inplace = True)
for ind in inds:
    wilson['{} Mantissa'.format(ind)] = mantissa_round(wilson['{}'.format(ind)])
wilson['County'] = 'Wilson County'

In [578]:
diff = wilson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Agriculture, forestry, fishing and hunting Diff', 'Mining, quarrying, and oil and gas extraction Diff', 'Utilities Diff', 'Construction Diff', 
              'Manufacturing Diff', 'Wholesale trade Diff', 'Retail trade Diff', 'Transportation and warehousing Diff', 'Information Diff', 'Finance and insurance Diff', 
              'Real estate and rental and leasing Diff', 'Professional, scientific, and technical services Diff', 'Management of companies and enterprises Diff', 
              'Administrative and support and waste management and remediation services Diff', 'Educational services Diff', 'Health care and social assistance Diff', 
              'Arts, entertainment, and recreation Diff', 'Accommodation and food services Diff', 'Other services (except government and government enterprises) Diff', 
              'Public Administration Diff']]
diff14 = diff.reset_index(drop = False)
diff14.head()

,County,"Agriculture, forestry, fishing and hunting Diff","Mining, quarrying, and oil and gas extraction Diff",Utilities Diff,Construction Diff,Manufacturing Diff,Wholesale trade Diff,Retail trade Diff,Transportation and warehousing Diff,Information Diff,Finance and insurance Diff,Real estate and rental and leasing Diff,"Professional, scientific, and technical services Diff",Management of companies and enterprises Diff,Administrative and support and waste management and remediation services Diff,Educational services Diff,Health care and social assistance Diff,"Arts, entertainment, and recreation Diff",Accommodation and food services Diff,Other services (except government and government enterprises) Diff,Public Administration Diff
0,Wilson County,0.0,-5.684342e-14,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [592]:
diffs = [diff1, diff2, diff3, diff4, diff5, diff6, diff7, diff8, diff9, diff10, diff11, diff12, diff13, diff14]
diff = pd.concat(diffs)

In [593]:
diff.to_csv('diffs.csv')

In [600]:
dfs = [cheatham, davidson, dickson, houston, humphreys, maury, montgomery, robertson, rutherford, stewart, sumner, trousdale, williamson, wilson]
data = pd.concat(dfs)

In [601]:
data['GEO_ID'] = data['GEO_ID'].astype(str)
#data['County'] = data['GEO_ID'].map(c_inverted)

In [602]:
data = data[['County', 'GEO_ID', 'Agriculture, forestry, fishing and hunting Mantissa', 'Mining, quarrying, and oil and gas extraction Mantissa', 'Utilities Mantissa',
             'Construction Mantissa', 
              'Manufacturing Mantissa', 'Wholesale trade Mantissa', 'Retail trade Mantissa', 'Transportation and warehousing Mantissa', 'Information Mantissa', 
             'Finance and insurance Mantissa', 
              'Real estate and rental and leasing Mantissa', 'Professional, scientific, and technical services Mantissa', 
             'Management of companies and enterprises Mantissa', 
              'Administrative and support and waste management and remediation services Mantissa', 'Educational services Mantissa', 
             'Health care and social assistance Mantissa', 
              'Arts, entertainment, and recreation Mantissa', 'Accommodation and food services Mantissa', 
             'Other services (except government and government enterprises) Mantissa', 
              'Public Administration Mantissa']]

In [605]:
data['GEO_ID'] = 'G'+ data['GEO_ID']

In [606]:
data.head()

,County,GEO_ID,"Agriculture, forestry, fishing and hunting Mantissa","Mining, quarrying, and oil and gas extraction Mantissa",Utilities Mantissa,Construction Mantissa,Manufacturing Mantissa,Wholesale trade Mantissa,Retail trade Mantissa,Transportation and warehousing Mantissa,Information Mantissa,Finance and insurance Mantissa,Real estate and rental and leasing Mantissa,"Professional, scientific, and technical services Mantissa",Management of companies and enterprises Mantissa,Administrative and support and waste management and remediation services Mantissa,Educational services Mantissa,Health care and social assistance Mantissa,"Arts, entertainment, and recreation Mantissa",Accommodation and food services Mantissa,Other services (except government and government enterprises) Mantissa,Public Administration Mantissa
0,Cheatham County,G470210701021001,0,0,1,8,1,1,1,1,1,1,1,1,0,1,1,96,1,1,1,24
1,Cheatham County,G470210701021004,0,0,0,21,0,0,0,0,0,0,0,18,0,0,0,1,0,0,0,0
2,Cheatham County,G470210701021009,0,0,0,2,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0
3,Cheatham County,G470210701021012,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0
4,Cheatham County,G470210701021013,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0


In [603]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12040 entries, 0 to 885
Data columns (total 22 columns):
 #   Column                                                                             Non-Null Count  Dtype 
---  ------                                                                             --------------  ----- 
 0   County                                                                             12040 non-null  object
 1   GEO_ID                                                                             12040 non-null  object
 2   Agriculture, forestry, fishing and hunting Mantissa                                12040 non-null  int32 
 3   Mining, quarrying, and oil and gas extraction Mantissa                             12040 non-null  int32 
 4   Utilities Mantissa                                                                 12040 non-null  int32 
 5   Construction Mantissa                                                              12040 non-null  int32 
 6   Manuf

In [607]:
data.to_csv('mantissadforadj.csv')